In [6]:
import pymysql

# 데이터베이스 연결
connection = pymysql.connect(
    host='localhost',
    user='bs',
    password='bs',
    database='bs_db'
)

cursor = connection.cursor()

In [7]:
# 테이블 생성
create_table_query = """
    CREATE TABLE IF NOT EXISTS customer(
        id INT AUTO_INCREMENT PRIMARY KEY,    
        username VARCHAR(50) NOT NULL,
        email VARCHAR(100) NOT NULL,
        age INT
    )
"""

cursor.execute(create_table_query)
connection.commit()

In [8]:
# 컬럼 추가
# 이 컬럼은 TIMESTAMP 데이터 타입을 가지며, 기본값으로 현재 타임스탬프(CURRENT_TIMESTAMP가 설정. 즉, 새로운 레코드가 삽입될 때 자동으로 현재 시간이 기록됩니다.)
alter_table_query = "ALTER TABLE customer ADD COLUMN created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP"
cursor.execute(alter_table_query)
connection.commit()

In [9]:
# email 컬럼에 NOT NULL 제약조건 제외
alter_table_query = "ALTER TABLE customer MODIFY email VARCHAR(100)"
cursor.execute(alter_table_query)
connection.commit()

In [5]:
# 테이블 삭제
drop_table_query = "DROP TABLE IF EXISTS customer"
cursor.execute(drop_table_query)
connection.commit()

In [10]:
# 데이터 입력
# %s는 mysql.connector 모듈에서 사용되는 플레이스홀더입니다. 이 플레이스 홀더는 데이터를 삽입할 때 사용됩니다.(sqlight ?)
insert_query = "INSERT INTO customer (username, email, age) VALUES (%s, %s, %s)"
data = ("john_doe", "john@example.com", 30)
cursor.execute(insert_query,data)
connection.commit()

In [17]:
# 데이터 조회
select_query = "SELECT * FROM customer"
cursor.execute(select_query)
results = cursor.fetchall()
for row in results :
    print(row)

(2, 'alice', 'alice@example.com', 25, datetime.datetime(2024, 7, 15, 12, 19, 24))
(3, 'bob', 'bob@example.com', 28, datetime.datetime(2024, 7, 15, 12, 19, 24))
(4, 'charlie', 'charlie@example.com', 22, datetime.datetime(2024, 7, 15, 12, 19, 24))


In [ ]:
# 연결 종료
cursor.close()
connection.close()

In [12]:
# 데이터 수정
update_query = "UPDATE customer SET age = %s WHERE username = %s"
cursor.execute(update_query, (31, "john_doe"))
connection.commit()

In [13]:
# 데이터 삭제
delete_query = "DELETE FROM customer WHERE username=%s"
cursor.execute(delete_query,'john_doe')
connection.commit()

In [15]:
# 데이터 반복 삽입 (각 항목은 리스트나 튜플과 같은 순서가 있는 데이터 구조이어야 합니다.)
customers = [
    ("alice","alice@example.com",25),
    ("bob","bob@example.com",28),
    ("charlie","charlie@example.com",22)
]

cursor.executemany(insert_query,customers)
connection.commit()

In [18]:
# 데이터 조회 결과를 DataFrame으로 변환
import pandas as pd

# 컬럼 이름 가져오기
column_names = [desc[0] for desc in cursor.description]

# DataFrame으로 변환
df = pd.DataFrame(results, columns=column_names)

print(df)

   id username                email  age          created_at
0   2    alice    alice@example.com   25 2024-07-15 12:19:24
1   3      bob      bob@example.com   28 2024-07-15 12:19:24
2   4  charlie  charlie@example.com   22 2024-07-15 12:19:24


In [19]:
# DataFrame를 CSV파일로 변환
df.to_csv('customers.csv', index=False)

In [20]:
# buyer 테이블 생성
create_table_query = """
    CREATE TABLE IF NOT EXISTS buyer (
        id INT AUTO_INCREMENT PRIMARY KEY,
        username VARCHAR(50) NOT NULL,
        email VARCHAR(100) NOT NULL,
        age INT
    )
"""

cursor.execute(create_table_query)
connection.commit()

In [22]:
# CSV 파일 읽어서 데이터베이스에 삽입
import csv

# CSV 파일 읽기 
file_path = 'customers.csv'
df = pd.read_csv(file_path)

# 필요한 컬럼만 선택
data_to_insert = df[['username','email','age']].values.tolist()

# SQL 쿼리 정의
insert_query = "INSERT INTO buyer (username, email, age) VALUES (%s, %s, %s)"   

# 데이터베이스에 삽입
cursor.executemany(insert_query,data_to_insert)
connection.commit()

In [ ]:
# 데이터 조회 및 DataFrame 변환
select_query = "SELECT * FROM buyer"
